In [ ]:
# Import libraries
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pypfopt import risk_models, expected_returns, EfficientFrontier

In [ ]:
# Load data from Yahoo Finance
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2024-10-31'

# Download data
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
data.columns = ['TSLA', 'BND', 'SPY']

# Plot the closing prices
data.plot(figsize=(12, 6), title='Historical Closing Prices')
plt.show()

In [ ]:




# Calculate daily returns
returns = data.pct_change().dropna()

# Annualized returns and covariance matrix
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

print("Annualized Mean Returns:\n", mean_returns)
print("\nAnnualized Covariance Matrix:\n", cov_matrix)

# Set initial portfolio weights
weights = np.array([0.4, 0.3, 0.3])

# Calculate portfolio performance
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, risk

# Objective function to maximize Sharpe Ratio
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    p_returns, p_risk = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_returns - risk_free_rate) / p_risk

# Constraints and bounds for optimization
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Weights sum to 1
bounds = tuple((0, 1) for _ in range(len(tickers)))

# Perform optimization to maximize the Sharpe Ratio
optimized_result = minimize(
    neg_sharpe_ratio, weights, args=(mean_returns, cov_matrix), method='SLSQP',
    bounds=bounds, constraints=constraints
)
optimal_weights = optimized_result.x

# Calculate optimized portfolio performance
optimized_return, optimized_risk = portfolio_performance(optimal_weights, mean_returns, cov_matrix)
sharpe_ratio = (optimized_return - 0.01) / optimized_risk

print("\nOptimized Portfolio Weights:", optimal_weights)
print("Expected Portfolio Return:", optimized_return)
print("Portfolio Volatility (Risk):", optimized_risk)
print("Sharpe Ratio:", sharpe_ratio)

# Visualize optimized portfolio performance
cumulative_returns = (1 + returns).cumprod()
portfolio_cumulative_returns = (cumulative_returns * optimal_weights).sum(axis=1)

# Plot cumulative returns for individual assets and optimized portfolio
plt.figure(figsize=(12, 6))
for ticker in tickers:
    plt.plot(cumulative_returns.index, cumulative_returns[ticker], label=f'{ticker} Cumulative Returns')
plt.plot(portfolio_cumulative_returns.index, portfolio_cumulative_returns, label='Optimized Portfolio', linewidth=2, color='black')
plt.title('Cumulative Returns - Optimized Portfolio vs Assets')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()
